In [3]:
import pymorphy2
import nltk
import ujson as json
import matplotlib.pyplot as plt
import numpy as np
import itertools
import gzip

from datetime import datetime

from collections import Counter

In [4]:
class Document:
    def __init__(self, init_dict):
        self.title = init_dict.get('title', '')
        self.description = init_dict.get('description', '')
        self.url = init_dict.get('url', '')
        self.site = init_dict.get('site', '')
        self.ts = datetime.fromtimestamp(init_dict['ts']) if 'ts' in init_dict else -1
    
    def __str__(self):
        res = ''
        res += 'url : %s\n' % self.url
        res += 'date : %s\n' % self.ts
        res += 'title : %s\n' % self.title
        res += 'description : %s\n' % self.description
        res += 'site : %s\n' % self.site
        return res

In [5]:
fin = gzip.open('dataset_mai.jsonl.gz')
for line in itertools.islice(fin, 10):
    data = json.loads(line.strip())
    print(Document(data))

url : http://bloknot-volzhsky.ru/news/volzhane-mogut-podat-zayavlenie-na-letnie-putevki-
date : 2019-11-30 18:26:10
title : Волжане могут подать заявление на летние путевки для детей
description : С понедельника заявления начинают принимать в МФЦ
site : bloknot-volzhsky.ru

url : https://trikky.ru/test-na-znanie-russkogo-yazyka-423354.html
date : 2019-11-30 18:26:48
title : 💗Тест на знание русского языка💗
description : Тест со сложными и легкими вопросами. Для кого-то будет легко набрать все 100 баллов, а кому-то будет немного тяжело. В любом случае попробовать стоит.1. Что изучает фразеология? способы образования слов устойчивые сочетания слов части речи2. На месте каких цифр в словах пишется н? В простенке между занавеше(1)ыми окнами были установле(2)ы часы, а рядом с ними […]
site : trikky.ru

url : https://topwar.ru/165315-chernomorskij-flot-popolnilsja-150-tonnym-plavkranom-proekta-02690.html
date : 2019-11-30 18:26:43
title : Черноморский флот пополнился 150-тонным плавкраном про

In [6]:
fin = gzip.open('dataset_mai.jsonl.gz')
dataset = []
dataset_test = []
for line in itertools.islice(fin, 10000):
    data = json.loads(line.strip())
    dataset.append(Document(data))
    
for line in itertools.islice(fin, 5000):
    data = json.loads(line.strip())
    dataset_test.append(Document(data))

In [7]:
test_doc = dataset[11]
print(test_doc)

url : https://trikky.ru/kakoy-ty-napitok-423256.html
date : 2019-11-30 18:26:48
title : Какой ты напиток?
description : Какой ты напиток?Зелёненький,свеженький мохито,или взрывной лимонад?А может быть,Кока Кола?Сейчас  и узнаем!1. Какой вкус в напитках тебе больше всего нравится? Яркие цвета - вот что самое главное! Эта многослойность напитков заставляет завораживаться... Сладость,я такая сладкоежка! Я обожаю газировку,такая взрывная,как шипучка!2. Какие типы цветов напитков тебе больше нравятся? Холодные(голубые,синие,фиолетовые разных оттенков) Чёрный/Белый/Белый в крапинку Зелёные,цвет природы. Яркие […]
site : trikky.ru



### Как поделить текст на слова?

In [8]:
def print_words(a_document, a_split_func):
    print('------TITLE----------')
    for word in a_split_func(test_doc.title):
        print(word)

    print('------DESCRIPTION----------')
    for word in a_split_func(test_doc.description):
        print(word)

In [9]:
### Побить по пробелам?
def split_words_v1(a_text):
    for word in a_text.split():
        yield word

print_words(test_doc, split_words_v1)

------TITLE----------
Какой
ты
напиток?
------DESCRIPTION----------
Какой
ты
напиток?Зелёненький,свеженький
мохито,или
взрывной
лимонад?А
может
быть,Кока
Кола?Сейчас
и
узнаем!1.
Какой
вкус
в
напитках
тебе
больше
всего
нравится?
Яркие
цвета
-
вот
что
самое
главное!
Эта
многослойность
напитков
заставляет
завораживаться...
Сладость,я
такая
сладкоежка!
Я
обожаю
газировку,такая
взрывная,как
шипучка!2.
Какие
типы
цветов
напитков
тебе
больше
нравятся?
Холодные(голубые,синие,фиолетовые
разных
оттенков)
Чёрный/Белый/Белый
в
крапинку
Зелёные,цвет
природы.
Яркие
[…]


In [10]:
### Побить по всем не буквам и цифрам?
def split_words_v2(a_text):
    cur_word = ''
    for letter in a_text:
        if letter.isalnum():
            cur_word += letter
        else:
            if cur_word: yield cur_word
            cur_word = ''
            
    if cur_word: yield cur_word
    
print_words(test_doc, split_words_v2)

------TITLE----------
Какой
ты
напиток
------DESCRIPTION----------
Какой
ты
напиток
Зелёненький
свеженький
мохито
или
взрывной
лимонад
А
может
быть
Кока
Кола
Сейчас
и
узнаем
1
Какой
вкус
в
напитках
тебе
больше
всего
нравится
Яркие
цвета
вот
что
самое
главное
Эта
многослойность
напитков
заставляет
завораживаться
Сладость
я
такая
сладкоежка
Я
обожаю
газировку
такая
взрывная
как
шипучка
2
Какие
типы
цветов
напитков
тебе
больше
нравятся
Холодные
голубые
синие
фиолетовые
разных
оттенков
Чёрный
Белый
Белый
в
крапинку
Зелёные
цвет
природы
Яркие


In [11]:
### Что делать с разными написаниями iphone6 и iphone 6 ? Как сделать чтобы всегда получать одну версию разбиения?
## Можно оставить студентам написать?
test_text = 'Где купить iPhone 6? и почему iPhone6 стоит так дорого?'
print(list(split_words_v1(test_text)))
print(list(split_words_v2(test_text)))

['Где', 'купить', 'iPhone', '6?', 'и', 'почему', 'iPhone6', 'стоит', 'так', 'дорого?']
['Где', 'купить', 'iPhone', '6', 'и', 'почему', 'iPhone6', 'стоит', 'так', 'дорого']


## Реализовать функцию которая единообразно будет разбивать такие строки?

In [12]:
def split_words_v3(a_text):
    cur_word = ''
    prev_is_alpha = False

    for letter in a_text:
        if  (letter.isalpha() and prev_is_alpha or 
            letter.isdigit() and not prev_is_alpha):
            cur_word += letter
        elif (letter.isalpha() and not prev_is_alpha or
             letter.isdigit() and prev_is_alpha):
            if cur_word: yield cur_word
            cur_word = letter
            prev_is_alpha = not prev_is_alpha
        else:
            if cur_word: yield cur_word
            cur_word = ''
            prev_is_alpha = False
    if cur_word: yield cur_word
         
print(list(split_words_v3(test_text)))
split_words_v3(test_text)

['Где', 'купить', 'iPhone', '6', 'и', 'почему', 'iPhone', '6', 'стоит', 'так', 'дорого']


<generator object split_words_v3 at 0x7fbbe5304f50>

In [13]:
words = list(split_words_v2(test_doc.title + ' ' + test_doc.description))

In [14]:
words

['Какой',
 'ты',
 'напиток',
 'Какой',
 'ты',
 'напиток',
 'Зелёненький',
 'свеженький',
 'мохито',
 'или',
 'взрывной',
 'лимонад',
 'А',
 'может',
 'быть',
 'Кока',
 'Кола',
 'Сейчас',
 'и',
 'узнаем',
 '1',
 'Какой',
 'вкус',
 'в',
 'напитках',
 'тебе',
 'больше',
 'всего',
 'нравится',
 'Яркие',
 'цвета',
 'вот',
 'что',
 'самое',
 'главное',
 'Эта',
 'многослойность',
 'напитков',
 'заставляет',
 'завораживаться',
 'Сладость',
 'я',
 'такая',
 'сладкоежка',
 'Я',
 'обожаю',
 'газировку',
 'такая',
 'взрывная',
 'как',
 'шипучка',
 '2',
 'Какие',
 'типы',
 'цветов',
 'напитков',
 'тебе',
 'больше',
 'нравятся',
 'Холодные',
 'голубые',
 'синие',
 'фиолетовые',
 'разных',
 'оттенков',
 'Чёрный',
 'Белый',
 'Белый',
 'в',
 'крапинку',
 'Зелёные',
 'цвет',
 'природы',
 'Яркие']

In [15]:
len(words)

74

In [16]:
uniq_words = Counter(words)
print("NumKeys : ", len(uniq_words))
print(uniq_words.most_common(10))

NumKeys :  63
[('Какой', 3), ('ты', 2), ('напиток', 2), ('в', 2), ('тебе', 2), ('больше', 2), ('Яркие', 2), ('напитков', 2), ('такая', 2), ('Белый', 2)]


## Могут ли быть разные написания одного слова?

In [17]:
def get_norm_word_v1(a_word):
    return a_word.lower()

uniq_words = Counter(get_norm_word_v1(word) for word in words)
print("NumKeys : ", len(uniq_words))
print(uniq_words.most_common(10))
### ('я', 2)

NumKeys :  62
[('какой', 3), ('ты', 2), ('напиток', 2), ('в', 2), ('тебе', 2), ('больше', 2), ('яркие', 2), ('напитков', 2), ('я', 2), ('такая', 2)]


In [18]:
from nltk.stem.snowball import RussianStemmer

In [19]:
stemmer = RussianStemmer()

In [20]:
stemmer.stem('Напитков')

'напитк'

In [21]:
def get_norm_word_v2(a_word):
    return stemmer.stem(a_word)

In [22]:
uniq_words = Counter(get_norm_word_v2(word) for word in words)
print("NumKeys : ", len(uniq_words))
print(uniq_words.most_common(10))
### ('я', 2)

NumKeys :  55
[('как', 5), ('напитк', 3), ('цвет', 3), ('ты', 2), ('напиток', 2), ('взрывн', 2), ('в', 2), ('теб', 2), ('больш', 2), ('нрав', 2)]


In [23]:
for word in words:
    if get_norm_word_v2(word) == 'как':
        print(word)

Какой
Какой
Какой
как
Какие


In [24]:
for word in words:
    if get_norm_word_v2(word) == 'цвет':
        print(word)

цвета
цветов
цвет


In [25]:
morth_analyzer = pymorphy2.MorphAnalyzer()

In [26]:
res = morth_analyzer.parse('Какая') #Какая , Напитков

In [27]:
res

[Parse(word='какая', tag=OpencorporaTag('ADJF,Apro femn,sing,nomn'), normal_form='какой', score=0.987012, methods_stack=((<DictionaryAnalyzer>, 'какая', 1676, 7),)),
 Parse(word='какая', tag=OpencorporaTag('GRND,impf,intr pres'), normal_form='какать', score=0.012987, methods_stack=((<DictionaryAnalyzer>, 'какая', 15, 67),))]

In [28]:
res[0].normal_form

'какой'

In [29]:
MORTH_CACHE = {}
def get_norm_word_v3(a_word):
    if a_word not in MORTH_CACHE: 
        MORTH_CACHE[a_word] = morth_analyzer.parse(a_word)[0].normal_form
        
    return MORTH_CACHE[a_word]

In [30]:
uniq_words = Counter(get_norm_word_v3(word) for word in words)
print("NumKeys : ", len(uniq_words))
print(uniq_words.most_common(10))
### ('напиток', 2)

NumKeys :  56
[('напиток', 5), ('какой', 4), ('ты', 4), ('взрывной', 2), ('в', 2), ('большой', 2), ('нравиться', 2), ('яркий', 2), ('я', 2), ('такой', 2)]


In [31]:
### v4 - числа преоразовывать к специальному формату YYYY , DD - чтобы уменьшить словарь и склеивались похожие названия отличающиеся только цифрами 

### Пример деревьев решений для предсказания сайта по tf-idf векторам
Получится ли обучить хорошее решение линейной моделью?

In [32]:
def get_doc_words(a_doc, a_split=split_words_v3, a_norm_word=get_norm_word_v3):
    for word in itertools.chain(a_split(a_doc.title), a_split(a_doc.description)):
        yield a_norm_word(word)

In [33]:
class NaiveBayes:
    
    min_class_c = 12
    
    def __init__(self):
        self.word_in_class = Counter()
        self.word_not_in_class = Counter()
        е
    def train(self, a_words, a_is_class):
        word_counter = self.word_in_class if a_is_class else self.word_not_in_class
        word_counter.update(a_words)
        
    def predict(self, a_words):
        res = 0
        num_words = 0
        for word in a_words:
            class_freq, not_class_freq = self.word_in_class.get(word, 0), self.word_not_in_class.get(word, 0)
            if class_freq < self.min_class_c or not_class_freq < self.min_class_c: continue
            num_words += 1
            res += np.log(class_freq/not_class_freq)
        return res/num_words if num_words > 0 else 0.0

In [34]:
dataset_sites = Counter(doc.site for doc in dataset)

sport_sites = set(['matchtv.ru', 'euro-football.ru', 'euro.com.ua',
                   'bobsoccer.ru', 'bombardir.ru', 'championat.com',
                   'football24.ru', 'rusfootball.info', 'soccernews.ru'
                  ])

for item in dataset_sites:
    if 'sport' in item: sport_sites.add(item)

In [107]:
for item in dataset_sites.most_common(1000):
    if item[0] not in sport_sites:
        print(item)
    
type(dataset_sites)

('tass.ru', 178)
('vk.com', 163)
('123ru.net', 143)
('gazeta.ru', 136)
('rossaprimavera.ru', 130)
('russian.rt.com', 118)
('daily-inform.ru', 108)
('newsyou.info', 95)
('365news.biz', 95)
('obozrevatel.com', 92)
('tvzvezda.ru', 83)
('moika78.ru', 78)
('life.ru', 73)
('wi-fi.ru', 71)
('riafan.ru', 71)
('rueconomics.ru', 69)
('pikabu.ru', 65)
('novosti-n.org', 63)
('mob-mobile.ru', 63)
('sm-news.ru', 58)
('news.ru', 58)
('mir24.tv', 57)
('nation-news.ru', 57)
('govoritmoskva.ru', 56)
('360tv.ru', 56)
('news-front.info', 53)
('terrikon.com', 53)
('vz.ru', 51)
('aif.ru', 51)
('dizainvfoto.ru', 50)
('argumenti.ru', 49)
('peterburg2.ru', 48)
('ukraina.ru', 46)
('oane.ws', 46)
('rosbalt.ru', 44)
('news.rambler.ru', 43)
('zelenj.ru', 40)
('ferra.ru', 39)
('bmwclub.ru', 39)
('habr.com', 39)
('deita.ru', 38)
('vestikavkaza.ru', 37)
('abnews.ru', 36)
('1rre.ru', 35)
('gazeta.spb.ru', 34)
('profile.ru', 34)
('fontanka.ru', 34)
('yerkramas.org', 33)
('riamo.ru', 33)
('radiosputnik.ria.ru', 32)
('ra

collections.Counter

In [36]:
sport_dataset = [item for item in dataset if item.site in sport_sites]
not_sport_dataset = [item for item in dataset if item.site not in sport_sites]
print(len(sport_dataset), len(not_sport_dataset))

1087 8913


In [37]:
%%time

bayes_classifier = NaiveBayes()
for item in sport_dataset:
    bayes_classifier.train(get_doc_words(item), a_is_class=True)
    
for item in not_sport_dataset:
    bayes_classifier.train(get_doc_words(item), a_is_class=False)

CPU times: user 20 s, sys: 27.6 ms, total: 20.1 s
Wall time: 20.1 s


In [38]:
for item in not_sport_dataset:
    predict = bayes_classifier.predict(get_doc_words(item))
    #if predict > np.log(len(sport_dataset)/len(not_sport_dataset)):
    if predict > np.log(len(sport_dataset)/len(not_sport_dataset)):
        print(predict, item)
#         break
        print('-*-'*13)


-1.9601938853085725 url : https://rusturkey.com/post/207586/grazhdanka-rossii-vypala-s-balkona-7-go-etazha-v-antale
date : 2019-11-30 18:26:11
title : Гражданка России выпала с балкона 7-го этажа в Анталье
description : Жизнь женщины не удалось спасти
site : rusturkey.com

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.837474331018305 url : https://riavrn.ru/news/voronezhskiy-batutist-stal-prizerom-chempionata-mira/
date : 2019-11-30 18:26:38
title : Воронежский батутист стал призером чемпионата мира
description : Сергей Азарян отличился в паре с Михаилом Мельником из Тольятти.
site : riavrn.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-2.0732243592395294 url : https://tengrinews.kz/events/krupnyiy-pojar-ohvatil-bannyiy-kompleks-u-podnojiya-kok-tobe-385419/
date : 2019-11-30 18:22:16
title : Крупный пожар охватил банный комплекс у подножия Кок-Тобе
description : Крупный пожар охватил банный комплекс "Царское село", расположенный у подножия горы Кок-Тобе в Алматы вечером в субботу, 30 ноября

-1.8868546777736355 url : http://milanac.ru/2019/pioli-popast-na-evrokubkovoe-mesto-yavlyaetsya-celyu/
date : 2019-11-30 18:47:40
title : Пиоли: «Попасть на еврокубковое место является целью»
description : Сможет ли «Милан» попасть в зону Лиги чемпионов? Пиоли: Наша цель заключается в том, чтобы думать о завтрашней игре. Мы понимаем, какая у нас позиция в турнирной таблице. Я позвал команду на ужин в среду для того, чтобы мы вместе посмотрели матчи Лиги чемпионов. Европа является целью для «Милана». Команда борется, играет и бежит, мы …
site : milanac.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.142928455781536 url : https://news.ru/others/rossiyane-zanyali-chetvyortoe-mesto-v-smeshannoj-estafete-na-km-po-biatlonu/
date : 2019-11-30 18:46:13
title : Россияне заняли четвёртое место в смешанной эстафете на КМ по биатлону
description : Российские биатлонисты заняли четвёртое место в смешанной эстафете на первом этапе Кубка мира в шведском Эстерсунде.
site : news.ru

-*--*--*--*--*--*--*

-1.1268161471460463 url : https://fighttime.ru/news/item/19382-oleg-taktarov-koklyaev-ispugalsya-i-upal.html
date : 2019-11-30 19:03:08
title : Ветеран смешанных единоборств, Олег Тактаров, подвел итоги матча по боксу между Александром Емельяненко и Михаилом Кокляевым, который завершился поражением тяжелоатлета нокаутом в первом раунде.
description : Ветеран смешанных единоборств, Олег Тактаров, подвел итоги матча по боксу между Александром Емельяненко и Михаилом Кокляевым, который завершился поражением тяжелоатлета нокаутом в первом раунде.
site : fighttime.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.7448975775011353 url : http://mixednews.ru/archives/157029
date : 2019-11-30 19:03:40
title : Концерн Volkswagen — официальный партнер чемпионата ЕВРО 2020, который будет обеспечивать мобильность участников, болельщиков и организаторов чемпионата
description : Внимание всех любителей футбола было недавно приковано к жеребьёвке финального раунда чемпионата Европы по футболу 2020 года, к

-1.2128297158348005 url : https://www.gazeta.ru/sport/results/england/2019-2020/12453355.shtml
date : 2019-11-30 19:11:11
title : Дерби Лондона: "Челси" пропустил от "Вест Хэма". LIVE
description : "Челси" в 14-м туре Английской премьер-лиги (АПЛ) на своем поле принимает "Вест Хэм Юнайтед". "Газета.Ru" ведет текстовую онлайн-трансляцию.
site : gazeta.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.476543458220448 url : https://tass.ru/sport/7234843
date : 2019-11-30 19:12:21
title : Россияне заняли 12-е место в сингл-миксте на этапе КМ по биатлону
description : Первое место заняла сборная Швеции
site : tass.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.2667731986814559 url : https://nation-news.ru/region/spb/488022-boyarskii-mostovoi-i-genich-ocenili-perspektivy-spartaka-i-zenita-v-matche-18-tura-rpl
date : 2019-11-30 19:11:12
title : Боярский, Мостовой и Генич оценили перспективы "Спартака" и "Зенита" в 18-м туре РПЛ
description : Популярный телекомментатор Константин Генич, экс-футбо

-1.8736712613144497 url : https://www.f1news.ru/news/f1-141436.html
date : 2019-11-30 19:36:43
title : Марио Изола: В гонке многое решит стратегия
description : Быстрейшей в завтрашней гонке в Pirelli считают стратегию одного пит-стопа...
site : f1news.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.22217680336824 url : https://russian.rt.com/sport/news/692387-cherchesov-zherebyovka-evro-2020
date : 2019-11-30 19:37:16
title : Черчесов поделился ожиданиями от жеребьёвки Евро-2020
description : Главный тренер сборной России по футболу Станислав Черчесов признался, что пока не изучал сборные Уэльса и Финляндии.
site : russian.rt.com

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.9525855678268256 url : https://newizv.ru/news/society/30-11-2019/aktivisty-proveli-vserossiyskuyu-aktsiyu-protiv-razvala-meditsiny
date : 2019-11-30 19:36:10
title : Активисты провели всероссийскую акцию против развала медицины
description : «#мыхотимжить». Под таким названием прошли акции в российских городах 30 ноя

...

-1.4610288301028753 url : https://portamur.ru/news/detail/v-tyinde-proizoshel-krupnyiy-pojar/
date : 2019-11-30 13:12:12
title : В Тынде произошел крупный пожар
description : Сгорел гаражный бокс с 20 грузовиками внутри
site : portamur.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.5581996816326327 url : https://intellifishing.ru/mesta/ufe
date : 2019-11-30 13:11:46
title : Рыбалка в Уфе
description : Особенности рыбалки в Уфе. Описание местных водоемов, какая рыба водится, выбор лучших мест для вылова, необходимые снасти и приманки.
site : intellifishing.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-0.9843581920570093 url : http://radiovesti.ru/news/1245562/
date : 2019-11-30 13:13:09
title : Слуцкий покинул пост главного тренера нидерландского «Витесса» Смена власти в ЕС: подведем итоги
description : Причиной стало неудачное выступление «орлов» в чемпионате Голландии.
site : radiovesti.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.5405606435193664 url : https://ngs55.ru/news/more/663

-1.7953929414427527 url : https://gtrk.tv/novosti/156196-salavat-yulaev-sygraet-finskim-yokeritom-ne-propustite-pryamoy-efir-mayake
date : 2019-11-30 10:29:40
title : «Салават Юлаев» сыграет с финским «Йокеритом» - не пропустите прямой эфир на Маяке!
description : В прямом эфире радиопрограммы «Хоккей на Маяке» матч будет комментировать ведущий Марат Казарян и спортивный эксперт, тренер следж-хоккейного клуба «Башкирские Пираты» Никита Козлов. По традиции, самых активных слушателей ждут призы и подарки. Настраивайтесь на волну 100,6 в Уфе, 106,7 – в Стерлитамаке, Кумертау и Салавате в понедельник 2 декабря в 18 часов 55 минут и слушайте «Хоккей на Маяке»!
site : gtrk.tv

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.4550268580935137 url : https://bk55.ru/news/article/160833/
date : 2019-11-30 10:30:13
title : На хоккейном матче чиновников и бизнесменов Бурков провел символическое стартовое вбрасывание
description : В хоккей сыграли в частности чиновники правительства области и мэрии.
site

-1.979586198473203 url : https://russian.rt.com/world/news/692161-volodin-ukraina-zheneva
date : 2019-11-30 10:46:31
title : Володин прокомментировал поведение делегации Украины в Женеве
description : Спикер Госдумы России Вячеслав Володин прокомментировал поведение украинской делегации в ходе выступления депутата нижней палаты российского парламента от Крыма Руслана Бальбека на форуме ООН в Женеве.
site : russian.rt.com

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-2.1040482319576816 url : https://56orb.ru/news/society/30-11-2019/v-aeroportu-orenburga-i-na-gosgranitse-usilen-sanitarnyy-kontrol
date : 2019-11-30 10:46:27
title : В аэропорту Оренбурга и на Госгранице усилен санитарный контроль
description : В оренбургском управлении Роспотребнадзора отмечают рост заболеваемости населения гриппом и ОРВИ. Санитарный контроль на Госгранице и в аэропорту усилен.
site : 56orb.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.4755892280185368 url : https://russian.rt.com/sport/news/692163-ovechkin-n

-1.606249779628694 url : http://terrikon.com/posts/341321
date : 2019-11-30 10:54:10
title : Бавария - Байер: смотреть онлайн-видеотрансляцию Бундеслиги
description : "Бавария" со сменой тренера одерживает одну разгромную победу за другой. На днях была повержена "Црвена Звезда" с теннисным счетом 0:6, причем на выезде. А сегодня мюнхенцы сыграют с другим коллективом Лиги чемпионов - "Байером".
site : terrikon.com

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-2.087276441147043 url : http://moi-goda.ru/chto-sluchilos/danko-obvinil-rotaru-v-zhelanii-zarabotat-millioni-na-rossiyanach
date : 2019-11-30 10:59:03
title : Данко обвинил Ротару в желании "заработать миллионы на россиянах"
description : Певец Александр Фадеев, выступающий под псевдонимом Данко, призвал наказать организаторов концертов Софии Ротару в России. Он возмущен тем, что певица собирается заработать миллионы на россиянах.
site : moi-goda.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.3948567237243523 url : https://stadium.ru/n

-1.2212512062956025 url : http://www.evening-kazan.ru/news/ak-bars-poterpel-pervoe-porazhenie-posle-semi-pobed-podryad.html
date : 2019-11-30 11:07:05
title : «Ак Барс» накануне вечером дома уступил «Сочи» в матче регулярного чемпионата КХЛ. Автором победной шайбы стал российский форвард Степан Старков. Отметим, что это поражение прервало победную серию «Ак Барса» из семи матчей. В следующей встрече казанцы примут московское «Динамо». На данный казанский клуб, несмотря на поражение, продолжает лидировать на Востоке (51 очко). View the discussion thread.
description : «Ак Барс» накануне вечером дома уступил «Сочи» в матче регулярного чемпионата КХЛ. Автором победной шайбы стал российский форвард Степан Старков. Отметим, что это поражение прервало победную серию «Ак Барса» из семи матчей. В следующей встрече казанцы примут московское «Динамо». На данный казанский клуб, несмотря на поражение, продолжает лидировать на Востоке (51 очко). View the discussion thread.
site : evening-kazan.ru



In [39]:
### Попробывать обучить random_forest

In [40]:
%%time
word2id = {}
for item in dataset:
    item.words = list(get_doc_words(item))
    for word in item.words:
        if word not in word2id: word2id[word] = len(word2id)

CPU times: user 1.66 s, sys: 19.9 ms, total: 1.67 s
Wall time: 1.68 s


In [41]:
id2word = dict( (v,k) for k,v in word2id.items())

In [42]:
len(id2word)

31067

In [43]:
def doc2vector(a_doc):
    res = [0] * len(word2id)
    if not hasattr(a_doc, 'words'): a_doc.words = get_doc_words(a_doc)
    for word in a_doc.words: 
        if word in word2id: #проверка для тестового сета
            res[word2id[word]] += 1 
    return res
        
type(doc2vector(sport_dataset[13]))

list

In [44]:
from sklearn.ensemble import RandomForestClassifier

In [45]:
# classifier = RandomForestClassifier(n_estimators=3, max_depth=3)
classifier = RandomForestClassifier(n_estimators=3)

In [46]:
X = []
y = []
for item in sport_dataset:
    X.append(doc2vector(item))
    y.append(1)
    
for item in not_sport_dataset:
    X.append(doc2vector(item))
    y.append(0)

In [47]:
classifier.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=3,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [106]:
# for i, weight in sorted(enumerate(classifier.feature_importances_), key=lambda x: x[1], reverse=True):
#     print(i, weight, id2word[i])
    
classifier.feature_importances_
l = [1, 2, 3, 9, 3]
for i, weight in sorted(enumerate(l), key = lambda x: x[1], reverse = True):
    print(i, weight)
    
l.append(7)
l

3 9
2 3
4 3
1 2
0 1


[1, 2, 3, 9, 3, 7]

In [49]:
classifier.predict_proba([doc2vector(sport_dataset[13])])

array([[0., 1.]])

In [50]:
for item in not_sport_dataset:
    vec = doc2vector(item)
    if classifier.predict([vec]):
        print(item)

url : https://prognozist.ru/news/headline/1968977-rossiya-zanyala-4-mesto-v-smeshannoy-estafete-v-estersunde.html
date : 2019-11-30 18:22:30
title : Россия заняла 4 место в смешанной эстафете в Эстерсунде
description : Во второй гонке биатлонного сезона Россия заняла 4-е место и причиной во многом стал штрафной круг на первом этапе: Юрлова-Перхт не справилась со стрельбой стоя. Победила в смешанной эстафете Италия, Норвегии не хватило до первого места 4 секунды. 1. Италия 2. Норвегия +4,1 3. Швеция +59,9 4. Россия +1 мин 44 сек Логинов финиширует четвертым. 1 минуту 44 секунды Россия проигрывает лидеру Live! 🔴https://t.co/BrwJZVPSo5 pic.twitter.com/onFUTbHE1q— Биатлон (@biathlon_se) November 30, 2019 Йоханнес Бё приложил максимум усилий, чтобы отыграть 12 секунд на последнем отрезке, но Виндиш удержал запас. Россия выступала в привычном составе (Юрлова-Перхт, Миронова, Малышко, Логинов), но сильный ветер внес свои коррективы в гонку. Здесь сборная пусть ошибалась уже не так много, но в

url : https://news.ru/football/oskorblyavshij-dzyubu-bolelshik-na-matche-sbornoj-rossii-obyasnil-svoj-postupok/
date : 2019-11-30 17:36:42
title : Оскорблявший Дзюбу болельщик на матче сборной России объяснил свой поступок
description : Один из болельщиков, скандировавших оскорбительные кричалки в адрес капитана сборной России Артёма Дзюбы в матче против Сан-Марино, рассказал, что это была всего лишь шутка.
site : news.ru

url : https://vringe.com/news/129978-rakhim-chakhkiev-brosil-perchatku-aleksandru-emelyanenko-otvechaesh-za-svoi-slova.htm
date : 2019-11-30 17:37:09
title : Рахим Чахкиев бросил перчатку Александру Емельяненко: Отвечаешь за свои слова?
description : Олимпийский чемпион 2008 года и бывший претендент на мировой титул Рахим Чахкиев (26-3, 19 КО) из РФ вызвал на бой бойца смешанного стиля Александра Емельяненко (28-7-1 в ММА) оттуда же. Об этом сообщила пресс-службе Федерации бокса России. В ночь на сегодня Емельяненко в 1-м раунде нокаутировал дебютанта Михаила Кокляев

url : https://wi-fi.ru/news/3BkB0JyGkj36-smi-uznali-kto-mozhet-smenit-slutskogo-v-vitesse
date : 2019-11-30 15:06:43
title : Уволенному Слуцкому уже нашли замену в "Витессе"?
description : Бывший футболист "Витесса" Эдвард Стюринг может возглавить команду.
site : wi-fi.ru

url : http://terrikon.com/posts/341326
date : 2019-11-30 11:59:10
title : Евро-2020: сегодня в Бухаресте состоится жеребьевка матчей финального турнира
description : Жеребьевка финального турнира Евро-2020 начнется сегодня в Бухаресте в 19.00 по киевскому времени
site : terrikon.com

url : https://lentachel.ru/news/2019/11/30/sapogi-napoleona-prodali-na-auktsione-za-117208.html
date : 2019-11-30 12:02:06
title : Сапоги Наполеона продали на аукционе за €117 208
description : Аукционный дом Drouot сообщил, что сапоги Наполеона были проданы за €117 208. Утверждается, что эту пару обуви французский император носил на острове Святой Елены, где находился в ссылке до самой смерти. Сам аукцион состоялся в столице Франции. Ст

In [51]:
#### Тестовые данные
sport_test_dataset = [item for item in dataset_test if item.site in sport_sites]
not_sport_test_dataset = [item for item in dataset_test if item.site not in sport_sites]
print(len(sport_test_dataset), len(not_sport_test_dataset))

391 4609


In [52]:
X_test = []
y_test = []
for item in sport_test_dataset:
    X_test.append(doc2vector(item))
    y_test.append(1)
    
for item in not_sport_test_dataset:
    X_test.append(doc2vector(item))
    y_test.append(0)
    
y_test_pred = classifier.predict(X_test)

In [53]:
import sklearn.metrics

In [54]:
sklearn.metrics.precision_recall_fscore_support(y_test, y_test_pred)

(array([0.9515549 , 0.59340659]),
 array([0.97591668, 0.41432225]),
 array([0.96358183, 0.48795181]),
 array([4609,  391]))

In [55]:
sklearn.metrics.confusion_matrix(y_test, y_test_pred)

array([[4498,  111],
       [ 229,  162]])

In [56]:
#sklearn.metrics.confusion_matrix?

## Gensim

In [57]:
### https://logic.pdmi.ras.ru/~sergey/slides/N16_DataFestLDA.pdf

In [58]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models import ldamulticore
from gensim.models import LdaModel

In [59]:
# Create a corpus from a list of texts
common_dictionary = Dictionary(item.words for item in dataset)

In [60]:
# common_dictionary.filter_extremes()

In [61]:
def show_top_words(a_dictionary, a_top_n=100):
    for i, (id_word, freq) in enumerate(Counter(a_dictionary.dfs).most_common(a_top_n)):
        print(i, a_dictionary[id_word], freq, sep='\t\t')

In [62]:
show_top_words(common_dictionary)

0		в		7595
1		на		4996
2		и		4793
3		с		3580
4		по		2463
5		не		2273
6		о		2267
7		что		2198
8		из		2004
9		это		1823
10		быть		1762
11		который		1728
12		за		1531
13		год		1527
14		он		1441
15		как		1382
16		для		1362
17		один		1233
18		россия		1182
19		свой		1172
20		к		1119
21		весь		1080
22		от		1076
23		ноябрь		966
24		а		944
25		стать		842
26		они		837
27		российский		820
28		но		815
29		мочь		811
30		у		800
31		тот		794
32		30		791
33		человек		758
34		матч		682
35		время		677
36		новый		668
37		после		657
38		она		653
39		до		648
40		этот		647
41		рассказать		647
42		москва		621
43		два		612
44		1		598
45		самый		578
46		сообщать		556
47		при		554
48		хороший		549
49		мы		542
50		такой		529
51		область		518
52		2		511
53		так		508
54		день		505
55		то		493
56		уже		493
57		мир		493
58		заявить		482
59		декабрь		477
60		под		463
61		главный		454
62		также		453
63		2019		448
64		сообщить		447
65		ещё		431
66		сайт		430
67		место		428
68		большой		427
69		только		421
70		или		414


In [63]:
# common_dictionary.filter_n_most_frequent(17)

In [64]:
def filter_short_words(words):
    for word in words:
        if len(word) >= 3: yield word

In [65]:
common_dictionary = Dictionary(filter_short_words(item.words) for item in dataset)

In [66]:
show_top_words(common_dictionary)

0		что		2198
1		это		1823
2		быть		1762
3		который		1728
4		год		1527
5		как		1382
6		для		1362
7		один		1233
8		россия		1182
9		свой		1172
10		весь		1080
11		ноябрь		966
12		стать		842
13		они		837
14		российский		820
15		мочь		811
16		тот		794
17		человек		758
18		матч		682
19		время		677
20		новый		668
21		после		657
22		она		653
23		этот		647
24		рассказать		647
25		москва		621
26		два		612
27		самый		578
28		сообщать		556
29		при		554
30		хороший		549
31		такой		529
32		область		518
33		так		508
34		день		505
35		уже		493
36		мир		493
37		заявить		482
38		декабрь		477
39		под		463
40		главный		454
41		также		453
42		2019		448
43		сообщить		447
44		ещё		431
45		сайт		430
46		место		428
47		большой		427
48		только		421
49		или		414
50		команда		410
51		видео		410
52		александр		410
53		можно		407
54		район		396
55		страна		394
56		пройти		390
57		какой		388
58		2020		388
59		чтобы		386
60		если		383
61		дело		379
62		сегодня		373
63		другой		366
64		работа		360
65		город		358
66		по

In [67]:
common_dictionary.filter_n_most_frequent(7)

In [68]:
common_corpus = [common_dictionary.doc2bow(item.words) for item in dataset]

In [69]:
# Train the model on the corpus.
# lda = ldamulticore.LdaMulticore(common_corpus, num_topics=10, workers=1)
lda = LdaModel(common_corpus, num_topics=100, id2word=common_dictionary)

In [70]:
lda.show_topics(50)

[(98,
  '0.038*"мясо" + 0.036*"факт" + 0.032*"услуга" + 0.028*"устроить" + 0.027*"база" + 0.024*"секрет" + 0.020*"поезд" + 0.020*"получаться" + 0.017*"мебель" + 0.015*"полуостров"'),
 (13,
  '0.024*"игра" + 0.024*"изменение" + 0.023*"тренировка" + 0.023*"начало" + 0.022*"день" + 0.020*"декабрь" + 0.019*"прогноз" + 0.018*"канал" + 0.017*"2019" + 0.016*"занятие"'),
 (71,
  '0.022*"ребёнок" + 0.020*"тысяча" + 0.017*"рубль" + 0.012*"200" + 0.011*"сайт" + 0.011*"размер" + 0.009*"квартира" + 0.009*"мвд" + 0.009*"источник" + 0.009*"москва"'),
 (93,
  '0.039*"россия" + 0.034*"власть" + 0.030*"русский" + 0.028*"заявить" + 0.018*"российский" + 0.018*"гражданин" + 0.018*"министр" + 0.016*"оказаться" + 0.016*"мужчина" + 0.015*"патрулирование"'),
 (84,
  '0.024*"голландский" + 0.022*"технический" + 0.021*"витессать" + 0.021*"здоровье" + 0.019*"высказаться" + 0.019*"социальный" + 0.019*"руководитель" + 0.019*"кино" + 0.017*"возраст" + 0.016*"слуцкий"'),
 (78,
  '0.028*"прямая" + 0.025*"судьба" + 0.0

In [71]:
# Compute Perplexity
print('Perplexity: ', lda.log_perplexity(common_corpus))  # a measure of how good the model is. lower the better.

Perplexity:  -20.480753156927328


In [72]:
doc_lda = lda[common_corpus]

In [73]:
for i,item in itertools.islice(enumerate(doc_lda), 100):
    print(item)
    print(dataset[i])

[(13, 0.055625293), (18, 0.13271315), (25, 0.07761717), (58, 0.34473222), (61, 0.093086645), (66, 0.22388928)]
url : http://bloknot-volzhsky.ru/news/volzhane-mogut-podat-zayavlenie-na-letnie-putevki-
date : 2019-11-30 18:26:10
title : Волжане могут подать заявление на летние путевки для детей
description : С понедельника заявления начинают принимать в МФЦ
site : bloknot-volzhsky.ru

[(23, 0.047750305), (27, 0.5400055), (37, 0.04040878), (42, 0.07276961), (47, 0.06953396), (50, 0.02861414), (55, 0.0826694), (88, 0.0976009)]
url : https://trikky.ru/test-na-znanie-russkogo-yazyka-423354.html
date : 2019-11-30 18:26:48
title : 💗Тест на знание русского языка💗
description : Тест со сложными и легкими вопросами. Для кого-то будет легко набрать все 100 баллов, а кому-то будет немного тяжело. В любом случае попробовать стоит.1. Что изучает фразеология? способы образования слов устойчивые сочетания слов части речи2. На месте каких цифр в словах пишется н? В простенке между занавеше(1)ыми окнами 

[(1, 0.12322045), (47, 0.21146417), (54, 0.19993608), (72, 0.1293667), (87, 0.11346195), (96, 0.11808651)]
url : http://bloknot-rostov.ru/news/gde-v-rostove-otklyuchat-svet-na-sleduyushchey-ned-1166562
date : 2019-11-30 18:26:40
title : Где в Ростове отключат свет на следующей неделе
description : График отключений – внутри
site : bloknot-rostov.ru

[(2, 0.6719554), (10, 0.047454674), (18, 0.11733919), (67, 0.0696221), (82, 0.05559718)]
url : https://nahnews.org/1014350-oficer-vsu-zastrelil-ukrainskogo-soldata-otkazavshegosya-voevat-protiv-donbassa
date : 2019-11-30 18:22:15
title : Офицер ВСУ застрелил украинского солдата, отказавшегося воевать против Донбасса
description : Солдаты Вооруженных сил Украины массово отказываются принимать участие в так называемой Операции объединенных сил против мирного
site : nahnews.org

[(15, 0.036534972), (28, 0.42860463), (34, 0.453489), (48, 0.0517373)]
url : https://foodman.club/recipe/salat-so-svininoj-i-marinovannymi-gribami/
date : 2019-11-30 1

In [74]:
# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
# trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# trigram_mod = gensim.models.phrases.Phraser(trigram)

# # See trigram example
# print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, common_corpus, common_dictionary)
vis

In [86]:
from gensim.models import word2vec

In [87]:
fin = gzip.open('dataset_mai.jsonl.gz')
dataset_w2v = []
for line in itertools.islice(fin, 100000):
    data = json.loads(line.strip())
    dataset_w2v.append(Document(data))

In [88]:
%%time
for item in dataset_w2v: item.words = list(get_doc_words(item))

CPU times: user 15.4 s, sys: 55.5 ms, total: 15.5 s
Wall time: 15.4 s


In [108]:
w2v = word2vec.Word2Vec([item.words for item in dataset_w2v], workers=4)

[['волжанин',
  'мочь',
  'подать',
  'заявление',
  'на',
  'летний',
  'путёвка',
  'для',
  'ребёнок',
  'с',
  'понедельник',
  'заявление',
  'начинать',
  'принимать',
  'в',
  'мфц'],
 ['тест',
  'на',
  'знание',
  'русский',
  'язык',
  'тест',
  'с',
  'сложный',
  'и',
  'лёгкое',
  'вопрос',
  'для',
  'кто',
  'то',
  'быть',
  'легко',
  'набрать',
  'весь',
  '100',
  'балл',
  'а',
  'кома',
  'то',
  'быть',
  'немного',
  'тяжело',
  'в',
  'любой',
  'случай',
  'попробовать',
  'стоить',
  '1',
  'что',
  'изучать',
  'фразеология',
  'способ',
  'образование',
  'слово',
  'устойчивый',
  'сочетание',
  'слово',
  'часть',
  'речь',
  '2',
  'на',
  'место',
  'какой',
  'цифра',
  'в',
  'слово',
  'писаться',
  'наш',
  'в',
  'простенок',
  'между',
  'занавеш',
  '1',
  'ыми',
  'окно',
  'быть',
  'установля',
  '2',
  'ы',
  'часы',
  'а',
  'ряд',
  'с',
  'они'],
 ['черноморский',
  'флот',
  'пополниться',
  '150',
  'тонный',
  'плавкран',
  'проект',
  '

In [90]:
w2v.most_similar('bmw') #

/home/max/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('nissan', 0.9066793918609619),
 ('volkswagen', 0.8897128105163574),
 ('toyota', 0.8820303678512573),
 ('hyundai', 0.8750872611999512),
 ('ford', 0.8659690618515015),
 ('купе', 0.8642557859420776),
 ('седан', 0.8631490468978882),
 ('тюнинг', 0.862923264503479),
 ('lada', 0.8628991842269897),
 ('lexus', 0.85258948802948)]

In [91]:
w2v['bmw']

/home/max/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.38653418,  0.5255957 ,  0.5784914 ,  1.0901593 , -0.8571623 ,
       -1.1789875 , -0.19142595,  0.86090344,  0.41520682, -1.9698603 ,
       -0.40049356,  0.16085343, -0.7476711 , -0.47296602,  0.54842836,
        0.7981237 , -1.4419955 , -2.0589898 , -0.7700965 , -0.554982  ,
        0.92203844, -0.43514284, -0.5190437 ,  0.40060738, -0.45462513,
       -0.43423226,  0.4558396 ,  0.06205593,  0.01674745, -0.06391081,
       -0.5408756 , -0.75321615, -1.1851549 ,  0.0106831 , -0.89661103,
       -1.0220299 ,  0.5236269 ,  1.0421689 , -0.30105498,  1.025147  ,
        0.01967702, -1.1993566 ,  1.2010205 , -0.8678188 ,  0.15480578,
        0.6176614 ,  1.5566751 , -0.08950721,  0.6155923 ,  0.8434239 ,
        1.2811767 , -0.9845457 ,  0.16101786,  0.6080083 , -0.80030704,
       -0.06471869,  0.6133712 , -0.20112684,  1.0250254 ,  0.88073754,
        0.11718655, -0.44727066, -0.6263447 , -0.6326086 ,  0.841806  ,
       -1.5468208 , -0.42929375, -0.055525  , -1.0171254 , -0.50

In [92]:
from scipy.spatial.distance import cosine

In [93]:
cosine(w2v['bmw'], w2v['bmw'])

/home/max/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


0.0

In [94]:
print(1 - cosine(w2v['bmw'], w2v['nissan']))

0.9066795110702515


/home/max/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


## ДЗ - реализовать поиск похожих документов по текстовым векторам и по word2vec векторам

In [95]:
def get_word_match_most_similar_docs(a_doc, a_dataset, a_top_n=20):
    pass

def get_tf_idf_most_similar_doc(a_doc, dataset):
    # для каждого документа строится вектор размерности словаря (аналогично random forest) и добавляется idf
    # далее cosine между документами
    pass

def get_w2v_most_similar_doc(a_doc, dataset):
    # считается средний вектор по всем словам (можно при усреднении учитывать idf)
    pass

In [96]:
doc_id = 13
for sim_func in [get_word_match_most_similar_docs, get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    sim_func(dataset[doc_id], dataset)